# Perform strategic environmental assessment 
Purpose: get average housing density, area of rangeslands impacted, area of each land cover type impacted for each scenario

In [1]:
import gdal
import geopandas as gpd
import pandas as pd
import fiona as fi
import matplotlib
import matplotlib.pyplot as plt
from osgeo import ogr
import json
import geojson
import os
from rasterstats import zonal_stats
import rasterio
import numpy as np
import time

### Function to calculate zonal stats
The outputs are the sum or mean of the input raster (environmental metric) for each "zone"

In [2]:
def calcZS(zones, dissField_all, dissField_region, rasterPath, statsList, categorical, calcAllRegions, areaField):
    ## rasterstats manual: https://pythonhosted.org/rasterstats/manual.html#zonal-statistics
    ### calculate zonal stats for each region
    ## dissolve
    zones_diss_region = zones.dissolve(by=dissField_region)
    ## calc zonal stats
    stats_region = zonal_stats(zones_diss_region, rasterPath, categorical = categorical, stats=statsList)
    ## add region as new key/value to list of dictionaries (stats_region output format)
    for counter, region in enumerate(zones_diss_region.index):
        stats_region[counter]["region"] = region
    ## convert to pd df
    stats_region_df = pd.DataFrame(stats_region)
    
    if calcAllRegions == True:
        ### calculate zonal stats across all regions
        ## add field for dissolving
        zones[dissField_all] = 1
        ## dissolve
        zones_diss_all = zones.dissolve(by=dissField_all)
        ## calc zonal stats
        stats_all = zonal_stats(zones_diss_all, rasterPath, categorical = categorical, stats=statsList)
        ## convert to pd df
        stats_all_df = pd.DataFrame(stats_all)
        ## Add "region" column, set value to "All"
        stats_all_df["region"] = "all"
        
        ## concat the two pd dfs
        stats_concat = pd.concat([stats_all_df,stats_region_df], axis = 0, sort=True) 
        
    else:
        stats_concat = stats_region_df
    
        ## Create table of area sums of all selected sites
        sumSelectedSitesByZone = zones.groupby([dissField_region])[areaField].sum().reset_index()

        ## change area column
        sumSelectedSitesByZone = sumSelectedSitesByZone.rename(columns = {"Area": "area_allSelSites_km2", dissField_region: "region"})[["region","area_allSelSites_km2"]]

        ## merge the two df
        stats_concat = stats_concat.merge(sumSelectedSitesByZone, how = "inner", left_on="region", right_on="region")

        ## calculate percentage
        #stats_concat["percent_selSites"] = stats_concat["Area"]/stats_concat['area_allSelSites_km2']
    
    return stats_concat

## Import datasets
### Selected Project Areas (SPAs)

In [5]:
infrastructureType = "tx_longHaul" ## selSites, tx, or tx_longHaul
mainDir = "C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\"

if infrastructureType == "selSites":
    ## selected sites folder
    allFCfolder = "C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\spatialDisaggregation\\selectedsites_cleaned_shp"
    allFCList = [file for file in os.listdir(allFCfolder) if file.endswith(".shp")]
    suffix = ".shp"
    suffix_solar = ".shp"
    regionField_RESOLVEZONE = "RESOLVE_ZO"
    
if infrastructureType == "tx":
    ## selected sites folder
    spDisaggFolder = os.path.join(mainDir,"spatialDisaggregation\\") #^^
    allFCfolder = os.path.join(spDisaggFolder, "LeastCostPath\\SD_LCP_122018\\SD_LCP_diss")    
    allFCList = [file for file in os.listdir(allFCfolder) if file.endswith(".shp")]
    suffix = "_copy_LCP_erasedBuffLine_diss.shp"
    suffix_solar = "_LCP_erasedBuffLine_diss.shp"
    regionField_RESOLVEZONE = "RESOLVE_ZO"
    
if infrastructureType == "tx_longHaul":
    ## selected sites folder
    #spDisaggFolder = os.path.join(mainDir,"spatialDisaggregation\\") #^^
    allFCfolder = os.path.join(mainDir, "dataCollection\\existingEnergyInfrastructure\\BLMRecentlyApprovedProjects")    
    allFCList = [file for file in os.listdir(allFCfolder) if file.endswith(".shp")]
    suffix = "_copy_LCP_erasedBuffLine_diss.shp"
    suffix_solar = "_LCP_erasedBuffLine_diss.shp"
    regionField_RESOLVEZONE = "RESOLVE_ZO"

### Environmental metric raster datasets

In [6]:
## housing density layer:
hd_path = r'C:\Users\Grace\Documents\TNC_beyond50\PathTo100\dataCollection\envImpactAssessment\housingDensity\housingDen2010_merged_raster_proj.tif'
## agricultural land
agLand_path = r'C:\Users\Grace\Documents\TNC_beyond50\PathTo100\dataCollection\envImpactAssessment\gapLC\gaplc_merged_proj_agLand.tif'
## all land cover types
agLand_allCat_path = r'C:\Users\Grace\Documents\TNC_beyond50\PathTo100\dataCollection\envImpactAssessment\gapLC\gaplc_merged_proj.tif'
## agricultural alnds, but reclassed
agLand_reclass_path = r'C:\Users\Grace\Documents\TNC_beyond50\PathTo100\dataCollection\envImpactAssessment\gapLC\gaplc_merged_proj_reclassAg.tif'
## rangelands 
rangelands_path = r'C:\Users\Grace\Documents\TNC_beyond50\PathTo100\dataCollection\envImpactAssessment\RangeGrd\nrirng_RANGE_NUM_proj.tif'

## Loop zonal stats through SPAs for each raster dataset
Change the environmental metric raster and run the below cells for each metric

In [ ]:
envMetric = "housingDensity" ## housingDensity, landCover, rangelands

### Lists of scenarios and technologies to loop over for each env metric raster dataset

In [8]:
## List scenarios
scenList = ["In-State x Capped Basecase",  "Full WECC x Capped Basecase", "Part WECC x Capped Basecase", \
                        "In-State x Capped highDER", "Full WECC x Capped highDER", "Part WECC x Capped highDER", \
                        "In-State x Capped lowBatt", "Full WECC x Capped lowBatt", "Part WECC x Capped lowBatt",\
                        "In-State BaseUsex Basecase", "Full WECC BaseUsex Basecase", "Part WECC BaseUsex Basecase", \
                         "In-State BaseUsex highDER", "Full WECC BaseUsex highDER", "Part WECC BaseUsex highDER", \
                         "In-State BaseUsex lowBatt",  "Full WECC BaseUsex lowBatt", "Part WECC BaseUsex lowBatt",\
                        "In-State xW2W No Cap Basecase",\
                        "Full WECC xW2W No Cap Basecase",\
                        "Part WECC xW2W No Cap Basecase",\
                         "In-State xW2W No Cap highDER", "Full WECC xW2W No Cap highDER", \
                         "In-State xW2W No Cap lowBatt", "Full WECC xW2W No Cap lowBatt"]

## technologies
techList = ["Geothermal", "Wind", "Solar"]

df_RESOLVEscenarios_total = pd.read_csv(os.path.join("C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\", "RESOLVEoutputs", "Results Summary Workbook_v11_20181214_total.csv"))

### Assumptions for Housing density

In [7]:
if envMetric == "housingDensity":
    
    ## raster file path
    rasterPath = hd_path

    ## stats to calculate
    statList = ['count', 'mean', 'min', 'max', 'median']

    ## whether or not the raster is categorical
    categoricalRaster = False

    ## whether to calculate the "all" in the "region" column
    calcAllRegions = True

    ## column order of final csv
    master_df_col_list = ["tech", "envCat", "scenario", "selSites", "region", 'selSitesExist', 'count', 'mean', 'min', 'max', 'median']

    ## empty dataframe of results
    master_df = pd.DataFrame(columns = master_df_col_list)

    resultsFileName = os.path.join("C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\envImpactAssessment\\",  "areaImpacted_" + infrastructureType + "_SG12_housingDensity_df.csv")

### Assumptions for land cover

In [21]:
if envMetric == "landCover":
    ## raster file path
    rasterPath = agLand_reclass_path #agLand_allCat_path

    ## stats to calculate
    statList = []

    ## whether or not the raster is categorical
    categoricalRaster = True

    ## whether to calculate the "all" in the "region" column
    ## don't need to calculate for all regions because this can be done on the df later (by adding across all zones)
    calcAllRegions = False

    ## column order of final csv
    master_df_col_list = ["tech", "envCat", "scenario", "selSites", "region", 'selSitesExist', "area_allSelSites_km2", 0, 553, 555, 556, 557]

    ## empty dataframe of results
    master_df = pd.DataFrame(columns = master_df_col_list)

    resultsFileName = os.path.join("C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\envImpactAssessment\\",  "areaImpacted_" + infrastructureType + "_SG13_gaplc_SG13_df.csv")

### Assumptions for rangelands

In [23]:
if envMetric == "rangelands":

    ## raster file path
    rasterPath = rangelands_path #agLand_allCat_path

    ## stats to calculate
    statList = []

    ## whether or not the raster is categorical
    categoricalRaster = True

    ## whether to calculate the "all" in the "region" column
    ## don't need to calculate for all regions because this can be done on the df later (by adding across all zones)
    calcAllRegions = False

    ## column order of final csv
    master_df_col_list = ["tech", "envCat", "scenario", "selSites", "region", 'selSitesExist', "area_allSelSites_km2", 0, 1,2,3,4,5,11,12,21,22,23,24,31,80,81,82]

    ## empty dataframe of results
    master_df = pd.DataFrame(columns = master_df_col_list)

    resultsFileName = os.path.join("C:\\Users\\Grace\\Documents\\TNC_beyond50\\PathTo100\\envImpactAssessment\\", "areaImpacted_" + infrastructureType + " _SG17_rangelands_df.csv")

In [25]:
start_time = time.time()
print(start_time)

if infrastructureType == "tx_longHaul":
    print("Working on long haul lines")
    for line in allFCList:
        ## read selected sites (shp) as geodataframe
        sp = gpd.read_file(os.path.join(allFCfolder, line))
        print("Working on " + line)

        if len(sp) > 0:

            ## apply Zonal stats for housing density
            zs_out = calcZS(zones = sp, dissField_all = "dissolve", dissField_region = "STATE", rasterPath = rasterPath, \
                            statsList = statList, categorical = categoricalRaster, calcAllRegions = calcAllRegions, areaField = "Area")

            ## apply Zonal stats for agland
            #agLand_all = calcZS(zones = sp, dissField_all = "gridcode", dissField_region = geography['regionField'], rasterPath = agLand_path, statsList = ['count', 'sum'])

            ## add technology
            zs_out["tech"] = "none"

            ## add scenario name
            zs_out["scenario"] = "none"

            ## add env cat
            zs_out["envCat"] = "none"

            ## add selected sites file name
            zs_out["selSites"] = line

            ## add flag for whether or not there were selected sites for this geography
            zs_out["selSitesExist"] = 1

            ## append area_df to master df
            master_df = pd.concat([master_df, zs_out], axis = 0, sort = True)

            ## reorder fields:
            master_df = master_df[master_df_col_list]

            print("Completed " + line)
            print(str((time.time() - start_time)/(60)) + " minutes")
    
else:
    for tech in techList:
        ## categories
        if tech == "Geothermal":
            catList = {"Cat1" : "geothermal_cat1b", "Cat2" : "geothermal_cat2f",\
                    "Cat3" : "geothermal_cat3", "Cat4": "geothermal_cat4"}
            suffixFinal = suffix

        if tech == "Wind":
            catList = {"Cat1" : "wind_0_03_nonEnv_r3_cat1b_singlepart_gt1km2","Cat2" : "wind_0_03_nonEnv_r3_cat2f_singlepart_gt1km2",\
                    "Cat3" : "wind_0_03_nonEnv_r3_cat3c_singlepart_gt1km2", "Cat4": "wind_0_03_nonEnv_r3_cat4_singlepart_gt1km2"}
            suffixFinal = suffix

        if tech == "Solar":
            catList = {"Cat1" : "solarPV_0_0_nonEnv_r1_cat1b_singlepart_gt1km2","Cat2" : "solarPV_0_0_nonEnv_r1_cat2f_singlepart_gt1km2",\
                    "Cat3" : "solarPV_0_0_nonEnv_r1_cat3c_singlepart_gt1km2", "Cat4": "solarPV_0_0_nonEnv_r1_cat4_singlepart_gt1km2"}
            suffixFinal = suffix_solar
        print("===================================")
        print(tech)

        for cat in catList:
            for scen in scenList:
                scen = scen.replace("x", cat)
                if scen in df_RESOLVEscenarios_total.columns: 
                    scenName_field = scen.replace(" ", "_").replace("-", "")                
                    print("")
                    geographyList = []
                    separator = "_" 
                    oos_RESOLVE_filename = separator.join([catList[cat], "PA", "OOS_RESOLVEZONE", "net", scenName_field, "selected"]) + suffixFinal
                    oos_STATE_filename = separator.join([catList[cat], "PA", "state", "net", scenName_field, "selected"]) + suffixFinal
                    instate_filename = separator.join([catList[cat], "PA", "CA_RESOLVEZONE", "net", scenName_field, "selected"]) + suffixFinal

                    if "W2W" in scenName_field and "InState" not in scenName_field:
                        ## append both state and in-state filenames to geoList
                        geographyList.append({"file": oos_STATE_filename, "regionField": "STATE"})
                        geographyList.append({"file": instate_filename, "regionField": regionField_RESOLVEZONE})
                        print("W2W scenario for " + oos_STATE_filename + " and " + instate_filename)

                    if "InState" in scenName_field:
                        ## append only state filename to geoList
                        geographyList.append({"file": instate_filename, "regionField": regionField_RESOLVEZONE})
                        print("InState scenario for " + instate_filename)

                    if any(txt in scenName_field for txt in ["Capped","BaseUseCat1"]) and "InState" not in scenName_field:
                        ## append both OOS RESOLVE ZONES and in-state filenames to geoList
                        geographyList.append({"file": oos_RESOLVE_filename, "regionField": regionField_RESOLVEZONE})
                        geographyList.append({"file": instate_filename, "regionField": regionField_RESOLVEZONE})
                        print("OOS RESOLVE scenario for " + oos_RESOLVE_filename + " and " + instate_filename)

                    ## loop through each element of geoList
                    for geography in geographyList:
                        ## if file is in the geodatabase and it has not already been analyzed
                        ## ex: "geothermal_cat1b_PA_CA_RESOLVEZONE_net_Part_WECC_Cat1_Capped_highDER_selectedcriticalHabitat_SG05" not in list of these in the master_df
                        if geography['file'] in allFCList: #and geography['file']+envData["envDataLabel"] not in (master_df["selSites"]+master_df['envData']).tolist():

                            ## read selected sites (shp) as geodataframe
                            sp = gpd.read_file(os.path.join(allFCfolder, geography['file']))

                            if len(sp) > 0:

                                ## apply Zonal stats for housing density
                                zs_out = calcZS(zones = sp, dissField_all = "dissolve", dissField_region = geography['regionField'], rasterPath = rasterPath, \
                                                statsList = statList, categorical = categoricalRaster, calcAllRegions = calcAllRegions, areaField = "Area")

                                ## apply Zonal stats for agland
                                #agLand_all = calcZS(zones = sp, dissField_all = "gridcode", dissField_region = geography['regionField'], rasterPath = agLand_path, statsList = ['count', 'sum'])

                                ## add technology
                                zs_out["tech"] = tech

                                ## add scenario name
                                zs_out["scenario"] = scenName_field

                                ## add env cat
                                zs_out["envCat"] = cat

                                ## add selected sites file name
                                zs_out["selSites"] = geography['file']

                                ## add flag for whether or not there were selected sites for this geography
                                zs_out["selSitesExist"] = 1

                                ## append area_df to master df
                                master_df = pd.concat([master_df, zs_out], axis = 0, sort = True)

                                ## reorder fields:
                                master_df = master_df[master_df_col_list]

                                print("Completed " + geography["file"])
                                print(str((time.time() - start_time)/(60)) + " minutes")

                            else:
                                ## Create single row dataframe with NA for region and area_km2 to indicate that there are no selected sites for that scenario/geography
                                zs_out = pd.DataFrame(data = {"tech": [tech], "envCat": [cat], "scenario": [scenName_field], \
                                                               "selSites": [geography['file']], "region": ["NA"], "selSitesExist": [0]})

                                ## append area_df to master df
                                master_df = pd.concat([master_df, zs_out], axis = 0)

                                ## reorder fields:
                                master_df = master_df[master_df_col_list]

                                print("***There are no selected sites for empty " + geography["file"])
                        else:
                            print("Not in gdb or already analyzed and in master_df: " + geography['file'])

                    print(str((time.time() - start_time)/(60)) + " minutes")

## reorder fields:
master_df = master_df[master_df_col_list]
master_df.to_csv(path_or_buf = resultsFileName, index = False)
                            
elapsed_time = (time.time() - start_time)/(60)
print("^^^^ Total time for completion: " + str(elapsed_time) + " minutes")

1551758663.4143205
Working on long haul lines
Working on BLM_transmission_line_pref_route_energy_b2h_76mBuff_state.shp


C:\Users\Grace\Anaconda3\lib\site-packages\rasterstats\io.py:294: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


Completed BLM_transmission_line_pref_route_energy_b2h_76mBuff_state.shp
0.02578161160151164 minutes
Working on BLM_transmission_line_pref_route_energy_gateway_south_v2_76mBuff_state.shp
Completed BLM_transmission_line_pref_route_energy_gateway_south_v2_76mBuff_state.shp
0.04541409413019816 minutes
Working on BLM_transmission_line_pref_route_energy_gateway_west_v2_76mBuff_state.shp
Completed BLM_transmission_line_pref_route_energy_gateway_west_v2_76mBuff_state.shp
0.056219605604807536 minutes
Working on BLM_transmission_line_pref_route_energy_southline_state.shp
Completed BLM_transmission_line_pref_route_energy_southline_state.shp
0.07480728228886922 minutes
Working on BLM_transmission_line_pref_route_energy_sunzia_123mBuff_state.shp
Completed BLM_transmission_line_pref_route_energy_sunzia_123mBuff_state.shp
0.11414523919423421 minutes
Working on BLM_transmission_line_pref_route_energy_transwest_express_centerline_76mBuff_state.shp
Completed BLM_transmission_line_pref_route_energy_trans